
# Ce notebook classe des phases en 3 catégories "cher", "moyen","éco".
C'est le premier d'une série de notebook pour comparer différentes techniques d'encodeing de textes et de classification.'

Keras uses tensorflow uses sklearn uses numpy and pandas

Tourne en un peu moins d'1 ms par epoch et donne un taux d'apprentissage "par coeur" de l'ordre de 70%.

Pas utile de le tester en généralisation sur des cas inconnus.

L'ensemble d'entrainement comprend 73 cas.

Le réseau est de type "sequential" à 3 couches.
cinq neurones en entrée
3 en sorties de type softmax


In [1]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import text_to_word_sequence
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder


Using TensorFlow backend.


 DATA PROCESS
 les données ont été saisies sous excel et sauvegardées en csv utf-8 bizarement avec l'option séparateur virgules de microsoft
 


In [2]:
def getDataset():
	dataset = pd.read_csv("RuleCout2.csv", delimiter=";", encoding='utf-8')
	#print ( 'dataset loaded with shape',dataset.shape)
	X = dataset['text']
	Y = dataset['class']
	dataset.set_index('text') #pour avoir les classes dans un ordre quelconque
	return dataset, X, Y

# utile pour les colonnes sur une ligne et afficher toutes les lignes
pd.set_option('display.width', 200)
pd.set_option('display.max_rows', 1000)

# on ajoutera peu à peu de nouvelles colonnes au dataset 
dataset, X, Y = getDataset()
print(dataset)

                         text  class
0             à prix modique     éco
1     aucun problème d'argent   cher
2                 avantageux     éco
3               bas de gamme     éco
4           bistrot pas cher     éco
5                         bon   cher
6                 bon marché     éco
7   bon rapport qualité prix   moyen
8                       cher    cher
9                         éco    éco
10                économique     éco
11             entre 10 et 20    éco
12             gastronomique    cher
13             haut de gamme    cher
14           j’ai les moyens    cher
15             je m’en fiche    cher
16              je m’en fous    cher
17             je suis riche    cher
18          jusqu'à cinquante   cher
19  le meilleur des meilleurs   cher
20                      luxe    cher
21                   luxueux    cher
22      ma bourse est pleine    cher
23     menu pas cher le midi     éco
24                   modique     éco
25       moins de cent euros    cher
2

#### Les mots les plus signifiants ont été classés par catégorie.

La catégorie "éco" indique les mots qui vont dans le sens de choix économiques
La catégorie "troquet" indique des classes de restaurant
La catégorie "négation" indique des mots qui inverse le sens
La catégorie "éco" indique les mots qui vont dans le sens de choix économiques
La catégorie "monnaie" indique des quantités monayables
La catégorie "ambigu" contient des termes qui se caractérisent par leur absence de positionnenement qui est une information en soit
Les catégories "nombres" sont explicites.

Les mots les plus signifiants vont permettre un encdage par catégorie.
ansi "pas trop cher" deviendra "négation 0 cher" ou [5 0 10]. 
On se posera la question de la pérsence du zéro : est-elle bénéfique ou néfaste ?

#### Quel est l'impact de ce travail de répartition sur  le taux de réussite ?



In [3]:
eco = ['modique','économique','ténu','japonais', 'chinois','kebab', 'self','pourri','avantageux','marché','bas',
       'plein','petit','petite']
troquet = ['troquet','bistrot','cantine','self','brasserie','restaurant','restau','resto']
negation = ['pas','ni','peu']
cher = ['cher','bon','haut','luxe','luxueux','fiche','fous','moyens', 'gastronomique', 'classe','riche','michelin','etoile','étoiles',
'standing','haut','chicos','onéreux','onereux','super''meilleur','meilleurs','grattin','gratin', 'bien']
monnaie = ['gamme', 'euro', 'prix', 'euros','problème','pb','qualité']
ambigu = ['milieu','moyen']
nombreEco = ['0','5','10','15','zéro','cinq','dix','quinze']
nombreCher= ['20','30','40','50','80','100','cent','vingt','trente','quarante','cinquante','quatre-vingt','cent']
limitatif = ['moins', 'peu','jusqu''à','plutôt']

# le choix de l'entier n'importe pas sauf pour le zéro qui est la valeur complémentée par pad_sequences
def score(word):
	if word in eco: 
		return 10
	elif word in nombreEco:
		return 20
	elif word in troquet: 
		return 25
	elif word in negation:
		return 40
	elif word in limitatif:
		return 40
	elif word in monnaie:
		return 50
	elif word in ambigu:
		return 51
	elif word in nombreCher:
		return 80
	elif word in cher:
		return 90
	else: 
		return 0
	
print ( 'score modique', score('modique'))
print ( 'score troquet', score('troquet'))
print ( 'score inconnu', score('inconnu'))


score modique 10
score troquet 25
score inconnu 0


In [4]:
#ai préféré utiliser le tokenizer de keras plutôt que celui de nltk car les valeurs filtrées par défaut comprennent l'apostrophe
def tokenize(sentence):
	tokens = text_to_word_sequence(sentence)
	return tokens
	
dataset['tokens']= dataset['text'].map(tokenize)
dataset['tokens']

0                 [à, prix, modique]
1        [aucun, problème, d'argent]
2                       [avantageux]
3                   [bas, de, gamme]
4               [bistrot, pas, cher]
5                              [bon]
6                      [bon, marché]
7      [bon, rapport, qualité, prix]
8                             [cher]
9                              [éco]
10                      [économique]
11               [entre, 10, et, 20]
12                   [gastronomique]
13                 [haut, de, gamme]
14               [j’ai, les, moyens]
15                 [je, m’en, fiche]
16                  [je, m’en, fous]
17                 [je, suis, riche]
18              [jusqu'à, cinquante]
19    [le, meilleur, des, meilleurs]
20                            [luxe]
21                         [luxueux]
22         [ma, bourse, est, pleine]
23       [menu, pas, cher, le, midi]
24                         [modique]
25          [moins, de, cent, euros]
26           [moins, de, dix, euros]
2

L'étape suivante consiste à transformer une liste de tokens en une liste de nombres à encoder.
Comme la taille de cette liste doit être fixe on utilise pad_sequence qui ajoute des zéros ou enlève des nombres. 

In [5]:
def token2code(tokens):
	return list(map(score,tokens))

#dataset['Xcodes'] = list (map(token2code,dataset['tokens']))
def encode(data):
	return dataset['tokens'].map(token2code)
	
		
Xcodes = encode(dataset)
XEncoded = pad_sequences(Xcodes,maxlen=5,padding='post')
dataset['XEncoded'] = XEncoded.tolist()
dataset


,text,class,tokens,XEncoded
0,à prix modique,éco,"[à, prix, modique]","[0, 50, 10, 0, 0]"
1,aucun problème d'argent,cher,"[aucun, problème, d'argent]","[0, 50, 0, 0, 0]"
2,avantageux,éco,[avantageux],"[10, 0, 0, 0, 0]"
3,bas de gamme,éco,"[bas, de, gamme]","[10, 0, 50, 0, 0]"
4,bistrot pas cher,éco,"[bistrot, pas, cher]","[25, 40, 90, 0, 0]"
5,bon,cher,[bon],"[90, 0, 0, 0, 0]"
6,bon marché,éco,"[bon, marché]","[90, 10, 0, 0, 0]"
7,bon rapport qualité prix,moyen,"[bon, rapport, qualité, prix]","[90, 0, 50, 50, 0]"
8,cher,cher,[cher],"[90, 0, 0, 0, 0]"
9,éco,éco,[éco],"[0, 0, 0, 0, 0]"


Le prochain pavé permet de transformer les catégories "éco","moyen" et cher en vecteur.
Ces vecteurs sont rangés dans la colonne 'Ycodes' du dataset

In [6]:
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)
dataset['Ycodes'] = pd.Series(list(dummy_y))
dataset


,text,class,tokens,XEncoded,Ycodes
0,à prix modique,éco,"[à, prix, modique]","[0, 50, 10, 0, 0]","[0.0, 0.0, 1.0]"
1,aucun problème d'argent,cher,"[aucun, problème, d'argent]","[0, 50, 0, 0, 0]","[1.0, 0.0, 0.0]"
2,avantageux,éco,[avantageux],"[10, 0, 0, 0, 0]","[0.0, 0.0, 1.0]"
3,bas de gamme,éco,"[bas, de, gamme]","[10, 0, 50, 0, 0]","[0.0, 0.0, 1.0]"
4,bistrot pas cher,éco,"[bistrot, pas, cher]","[25, 40, 90, 0, 0]","[0.0, 0.0, 1.0]"
5,bon,cher,[bon],"[90, 0, 0, 0, 0]","[1.0, 0.0, 0.0]"
6,bon marché,éco,"[bon, marché]","[90, 10, 0, 0, 0]","[0.0, 0.0, 1.0]"
7,bon rapport qualité prix,moyen,"[bon, rapport, qualité, prix]","[90, 0, 50, 50, 0]","[0.0, 1.0, 0.0]"
8,cher,cher,[cher],"[90, 0, 0, 0, 0]","[1.0, 0.0, 0.0]"
9,éco,éco,[éco],"[0, 0, 0, 0, 0]","[0.0, 0.0, 1.0]"


###MODEL DEFINITION
Le réseau comprend trois couches.
La première couche doit correspondre à la taille du vecteur X i.e au nombre de dimensions ici c'est cinq.
La couche de sortie comprend 3 neurons avec une fonction d'activation softmax, cette répartition entre les trois poids indiquent une probabilité d'appartenir à une des trois classes.
La taille de couche du milieu est ici de 11 avec une fonction d'activation Relu.


In [7]:
########################### MODEL DEFINITION
def sequential_model():
	# create model
	model = Sequential()
	model.add(Dense(5, input_dim=5, activation='relu'))
	model.add(Dense(3, activation='softmax'))
	# Compile model
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model

model = sequential_model()
model


In [8]:
###### TRAINING + EVALUATION
x_train = XEncoded
y_train = dummy_y
batch_size = 1
model.fit(x_train, y_train,
          epochs=200,
          batch_size=batch_size)
x_test = x_train
y_test = y_train
loss,metric = model.evaluate(x_test, y_test, batch_size=batch_size)
print ("Loss = ", loss," Accuracy = ",metric)

Epoch 1/200
73/73 [==============================] - 0s 6ms/step - loss: 5.9437 - acc: 0.5205
Epoch 2/200
73/73 [==============================] - 0s 1ms/step - loss: 5.1679 - acc: 0.5479
Epoch 3/200
73/73 [==============================] - 0s 1ms/step - loss: 4.7100 - acc: 0.6027
Epoch 4/200
73/73 [==============================] - 0s 2ms/step - loss: 4.6463 - acc: 0.6438
Epoch 5/200
73/73 [==============================] - 0s 1ms/step - loss: 4.6546 - acc: 0.6027
Epoch 6/200
73/73 [==============================] - 0s 1ms/step - loss: 4.6055 - acc: 0.5890
Epoch 7/200
73/73 [==============================] - 0s 1ms/step - loss: 4.6026 - acc: 0.6301
Epoch 8/200
73/73 [==============================] - 0s 1ms/step - loss: 4.6052 - acc: 0.5890
Epoch 9/200
73/73 [==============================] - 0s 1ms/step - loss: 4.5952 - acc: 0.6438
Epoch 10/200
73/73 [==============================] - 0s 1ms/step - loss: 4.6177 - acc: 0.5616
Epoch 11/200
73/73 [==============================] - 0s 1m

73/73 [==============================] - 0s 2ms/step - loss: 4.5266 - acc: 0.6849
Epoch 88/200
73/73 [==============================] - 0s 1ms/step - loss: 4.5340 - acc: 0.6849
Epoch 89/200
73/73 [==============================] - 0s 1ms/step - loss: 4.5353 - acc: 0.6849
Epoch 90/200
73/73 [==============================] - 0s 1ms/step - loss: 4.5335 - acc: 0.6849
Epoch 91/200
73/73 [==============================] - 0s 1ms/step - loss: 4.5326 - acc: 0.6849
Epoch 92/200
73/73 [==============================] - 0s 1ms/step - loss: 4.5239 - acc: 0.6849
Epoch 93/200
73/73 [==============================] - 0s 1ms/step - loss: 4.5273 - acc: 0.6712
Epoch 94/200
73/73 [==============================] - 0s 1ms/step - loss: 4.5277 - acc: 0.6712
Epoch 95/200
73/73 [==============================] - 0s 1ms/step - loss: 4.5230 - acc: 0.6986
Epoch 96/200
73/73 [==============================] - 0s 1ms/step - loss: 4.5028 - acc: 0.6986
Epoch 97/200
73/73 [==============================] - 0s 1ms/st

73/73 [==============================] - 0s 1ms/step - loss: 4.5168 - acc: 0.6849
Epoch 173/200
73/73 [==============================] - 0s 1ms/step - loss: 4.5062 - acc: 0.6986
Epoch 174/200
73/73 [==============================] - 0s 1ms/step - loss: 4.5001 - acc: 0.6712
Epoch 175/200
73/73 [==============================] - 0s 1ms/step - loss: 4.5136 - acc: 0.6849
Epoch 176/200
73/73 [==============================] - 0s 1ms/step - loss: 4.5091 - acc: 0.6986
Epoch 177/200
73/73 [==============================] - 0s 1ms/step - loss: 4.5088 - acc: 0.6986
Epoch 178/200
73/73 [==============================] - 0s 2ms/step - loss: 4.5009 - acc: 0.6986
Epoch 179/200
73/73 [==============================] - 0s 1ms/step - loss: 4.4963 - acc: 0.6849
Epoch 180/200
73/73 [==============================] - 0s 1ms/step - loss: 4.5127 - acc: 0.6986
Epoch 181/200
73/73 [==============================] - 0s 1ms/step - loss: 4.5024 - acc: 0.6986
Epoch 182/200
73/73 [==============================] -

In [9]:
print ("Loss = ", loss," Accuracy = ",metric)

Loss =  4.489910629712903  Accuracy =  0.6986301369863014


In [10]:
dataset['predicted'] = list(model.predict(XEncoded)) 

def decodeY(L):
	if L[0]>L[1] and L[0]>L[2]: 
		return "cher"
	elif L[2]>L[1] and L[2]>L[0]:
		return "éco"
	else:
		return "moyen"
	
dataset['resultat']=list(map(decodeY,dataset['predicted']))
dataset


,text,class,tokens,XEncoded,Ycodes,predicted,resultat
0,à prix modique,éco,"[à, prix, modique]","[0, 50, 10, 0, 0]","[0.0, 0.0, 1.0]","[1.1414263e-22, 0.0, 1.0]",éco
1,aucun problème d'argent,cher,"[aucun, problème, d'argent]","[0, 50, 0, 0, 0]","[1.0, 0.0, 0.0]","[5.77611e-23, 0.0, 1.0]",éco
2,avantageux,éco,[avantageux],"[10, 0, 0, 0, 0]","[0.0, 0.0, 1.0]","[0.039288215, 1.5219674e-08, 0.96071184]",éco
3,bas de gamme,éco,"[bas, de, gamme]","[10, 0, 50, 0, 0]","[0.0, 0.0, 1.0]","[0.17426275, 2.85665e-09, 0.8257373]",éco
4,bistrot pas cher,éco,"[bistrot, pas, cher]","[25, 40, 90, 0, 0]","[0.0, 0.0, 1.0]","[6.0085183e-12, 0.0, 1.0]",éco
5,bon,cher,[bon],"[90, 0, 0, 0, 0]","[1.0, 0.0, 0.0]","[0.99985003, 0.0, 0.00014990142]",cher
6,bon marché,éco,"[bon, marché]","[90, 10, 0, 0, 0]","[0.0, 0.0, 1.0]","[9.750895e-05, 0.0, 0.9999025]",éco
7,bon rapport qualité prix,moyen,"[bon, rapport, qualité, prix]","[90, 0, 50, 50, 0]","[0.0, 1.0, 0.0]","[7.9950805e-05, 0.0, 0.99992]",éco
8,cher,cher,[cher],"[90, 0, 0, 0, 0]","[1.0, 0.0, 0.0]","[0.99985003, 0.0, 0.00014990142]",cher
9,éco,éco,[éco],"[0, 0, 0, 0, 0]","[0.0, 0.0, 1.0]","[0.110838346, 0.006152227, 0.88300943]",éco


# To do : faire un graphique qui montre avec des couleurs différentes le positionnement des phrases, l'objectif serait d'avoir de mettre en évidence une capacité à séparer clairement les trois ensembles et cela avant toute tentative de généralisation